In [4]:
from FlagEmbedding import FlagModel
import numpy as np
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pprint import pprint as pp
import time
import umap
import os
import random
import time
from contextlib import contextmanager
from datasets import load_dataset, load_from_disk
from datasets import load_dataset, concatenate_datasets, load_from_disk
import pandas as pd
import datasets
from datasets import Dataset
from pprint import pprint as pp
from datasets import Dataset
from sklearn.cluster import KMeans
from tqdm import tqdm
import torch
import heapq

/home/tiger/miniconda3/envs/llm/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [5]:
model = FlagModel('BAAI/bge-large-en-v1.5', 
                  query_instruction_for_retrieval="",
                  use_fp16=True)

In [6]:
code_data = load_dataset("sahil2801/CodeAlpaca-20k")["train"]
fin_data = load_dataset("FinGPT/fingpt-sentiment-train")["train"]
med_data = load_dataset("medalpaca/medical_meadow_medical_flashcards")["train"]
general_data = load_dataset("tatsu-lab/alpaca")["train"]
math_data = load_dataset("TIGER-Lab/MathInstruct")["train"]
def alpaca_format(example):
    if example['input'] == "":
        example["instruction"] = example["instruction"]
    else:
        example["instruction"] = example["instruction"] + " " + example['input']
    example["response"] = example['output']
    return example

def process_sft_dataset(dataset_name, dataset, dataset_sample=None)->datasets.Dataset:
    if dataset_name in ["lucasmccabe-lmi/CodeAlpaca-20k", "yahma/alpaca-cleaned", "FinGPT/fingpt-sentiment-train"]:
        dataset = dataset.map(alpaca_format, remove_columns=['input', 'output'], desc=f"Preprocessing {dataset_name} for unified format.")
    elif dataset_name in ["WizardLM/WizardLM_evol_instruct_70k"]:
        dataset = dataset.rename_column("output", "response")
    elif dataset_name in ["tatsu-lab/alpaca", "vicgalle/alpaca-gpt4", "gbharti/finance-alpaca"]:
        dataset = dataset.map(alpaca_format, remove_columns=['input', 'output', 'text'], desc=f"Preprocessing {dataset_name} for unified format.")
    elif dataset_name in ["TIGER-Lab/MathInstruct"]:
        df = pd.DataFrame(dataset)
        df = df.drop_duplicates(subset=['instruction'])
        dataset = datasets.Dataset.from_pandas(df)
        dataset = dataset.rename_column("output", "response")
        dataset = dataset.remove_columns(['source'])
    elif dataset_name in ["lighteval/MATH"]:
        dataset = dataset.rename_column("solution", "response")
        dataset = dataset.rename_column("problem", "instruction")
        dataset = dataset.remove_columns(['level', 'type'])
    elif dataset_name in ['gsm8k']:
        dataset = dataset.rename_column("question", "instruction")
        dataset = dataset.rename_column("answer", "response")
    elif dataset_name in ['medalpaca/medical_meadow_medical_flashcards']:       # TODO: 'lavita/ChatDoctor-HealthCareMagic-100k'. not sure whether to discard the instruction.
        dataset = dataset.remove_columns(['instruction'])
        dataset = dataset.rename_column("input", "instruction")
        dataset = dataset.rename_column("output", "response")
    elif "math" in dataset_name:
        dataset = dataset.remove_columns(['source'])
        dataset = dataset.rename_column("output", "response")
    else:
        raise NotImplementedError(f"Dataset {dataset_name} is not supported.")
    dataset = dataset.shuffle(seed=2023)
    if dataset_sample:
        num_sample = min(len(dataset), dataset_sample)
        dataset = dataset.select(range(num_sample))
    print(f">> ===== After processing, Dataset {dataset_name} has {len(dataset)} examples. =====")
    return dataset
processed_data = []
for name, dataset in zip(["lucasmccabe-lmi/CodeAlpaca-20k","FinGPT/fingpt-sentiment-train","medalpaca/medical_meadow_medical_flashcards","tatsu-lab/alpaca","TIGER-Lab/MathInstruct"],[code_data,fin_data,med_data,general_data,math_data]):
# for name, dataset in zip(["lucasmccabe-lmi/CodeAlpaca-20k","FinGPT/fingpt-sentiment-train","medalpaca/medical_meadow_medical_flashcards", "TIGER-Lab/MathInstruct"],[code_data,fin_data,med_data,math_data]):
    tmp:datasets.Dataset = process_sft_dataset(name,dataset)
    print(tmp.column_names)
    processed_data.append(tmp)

>> ===== After processing, Dataset lucasmccabe-lmi/CodeAlpaca-20k has 20022 examples. =====
['instruction', 'response']
>> ===== After processing, Dataset FinGPT/fingpt-sentiment-train has 76772 examples. =====
['instruction', 'response']
>> ===== After processing, Dataset medalpaca/medical_meadow_medical_flashcards has 33955 examples. =====
['instruction', 'response']
>> ===== After processing, Dataset tatsu-lab/alpaca has 52002 examples. =====
['instruction', 'response']
>> ===== After processing, Dataset TIGER-Lab/MathInstruct has 224567 examples. =====
['response', 'instruction', '__index_level_0__']


In [76]:
domain_data = processed_data[4]

In [77]:
# niid code 构造
k=100
from sklearn.cluster import KMeans
import random

code_embeddings = model.encode(domain_data["instruction"])
# 假设 embeddings 是你的嵌入数据
kmeans = KMeans(n_clusters=k, random_state=0).fit(code_embeddings)
labels = kmeans.labels_
# 为每个簇创建一个数据集列表
local_datasets = []
sampled_idxs = []

# 随机选择 10 个簇
# random_clusters = random.sample(range(100),100)
# print(random_clusters)

# for i in random_clusters:
#     # 找到属于当前簇的所有数据点索引
#     cluster_indices = np.concatenate([np.where(labels == i)[0]],axis=None)
#     # 从当前簇中随机采样 10 个数据点
#     sampled_indices = np.random.choice(cluster_indices, size=min(10, len(cluster_indices)), replace=False)
#     sampled_idxs.extend(sampled_indices)
#     # 创建一个新的数据集，只包含采样的数据点
#     dataset = domain_data.select(sampled_indices)
#     # 将数据集添加到列表中
#     local_datasets.append(dataset)

unique_labels, label_counts = np.unique(labels, return_counts=True)
label_vocab = {label: idx for idx, label in enumerate(unique_labels)}
from partitions import *
from numpy import random

Chunks: 100%|██████████| 4/4 [00:42<00:00, 10.54s/it]


[54, 40, 71, 26, 49, 98, 28, 86, 68, 15, 8, 56, 45, 76, 38, 65, 59, 99, 85, 31, 22, 80, 94, 2, 53, 1, 35, 73, 81, 33, 23, 43, 0, 52, 79, 84, 64, 21, 3, 96, 48, 9, 67, 6, 55, 30, 51, 92, 89, 25, 87, 57, 93, 82, 37, 19, 32, 5, 16, 74, 18, 34, 27, 36, 13, 77, 39, 70, 58, 42, 7, 10, 72, 69, 90, 17, 88, 50, 44, 66, 91, 61, 29, 63, 97, 14, 46, 83, 60, 24, 20, 75, 95, 12, 11, 78, 4, 47, 62, 41]


In [47]:
# 定义生成NIID数据的函数
def generate_niid_data(code_embeddings, labels, client_num=10, alpha=1.0, seed=42):
    # 将标签转换为从0开始的索引
    label_assignment = np.array([label_vocab[label] for label in labels])
    
    # 调用 label_skew_process 生成分区结果
    partition_result = label_skew_process(
        label_vocab=label_vocab,
        label_assignment=label_assignment,
        client_num=client_num,
        alpha=alpha,
        data_length=len(labels),
        seed=seed
    )
    
    # 返回分区结果和原始标签
    return partition_result, labels

In [78]:
import pandas as pd
domain = "math"
alphas = [10,1,0.1,0.01]
client_num=100
seeds= [42,43,44]

for seed in seeds:
    for alpha in alphas:
        partition_result, labels = generate_niid_data(code_embeddings, labels, client_num=client_num, alpha=alpha, seed=seed)
        # 假设 code_data 是包含所有数据的DataFrame
        client_data = {i: [] for i in range(client_num)}

        for client_id, indices in enumerate(partition_result):
            # 根据分区结果中的索引提取对应的数据
            client_data[client_id] = domain_data.select(indices)

        # 将每个客户端的数据保存为Parquet格式
        if client_num==10:
            for client_id, data in client_data.items():
                data.save_to_disk(f'/mnt/bn/merlin-datavolume-tsy/leon/datasets/fed_data/niid_{domain}_{alpha}_{client_id}_{seed}.parquet')
        else:
            for client_id, data in client_data.items():
                data.save_to_disk(f'/mnt/bn/merlin-datavolume-tsy/leon/datasets/fed_data/niid_{domain}_{alpha}_{client_num}_{client_id}_{seed}.parquet')

client_num 100
[0.006862094608735923, 0.012530781459430816, 0.0096185102886889, 0.004795896102276826, 0.010642703513873366, 0.011203783280713552, 0.012655465852061967, 0.002556030048938624, 0.009845614003838498, 0.010032640592785226, 0.011101363958195104, 0.01674778573877729, 0.008442914586738034, 0.012268053632100888, 0.006198595519377291, 0.013385760151758718, 0.014387688306830479, 0.008665565287865091, 0.011528853304359056, 0.02101377317237172, 0.009841160989815956, 0.010794105990639765, 0.008006519212529, 0.011154800126465598, 0.004720194863893626, 0.009952486340379487, 0.006483588416819924, 0.020034110087412665, 0.015349539335699368, 0.012441721178979993, 0.002841022946381258, 0.005187761336260448, 0.00727622491283225, 0.009631869330756522, 0.007570123838319967, 0.011519947276313973, 0.008825873792676573, 0.009761006737410215, 0.011876188398117266, 0.007298489982944956, 0.002979066381080034, 0.006960060917231828, 0.006911077762983876, 0.01611991076159899, 0.012646559824016886, 0.0

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2312 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
# 计算每个客户端的每个标签的数量
client_num=10
client_label_counts = [[0 for _ in range(len(label_vocab))] for _ in range(client_num)]
for client_id, indices in enumerate(partition_result):
    client_labels = labels[np.array(indices)]
    for label in client_labels:
        client_label_counts[client_id][label] += 1

# 将计数转换为DataFrame
client_label_df = pd.DataFrame(client_label_counts, columns=label_vocab.values())
import seaborn as sns
import matplotlib.pyplot as plt

# 绘制热力图
plt.figure(figsize=(10, 10))  # 可以根据需要调整图形大小
sns.heatmap(client_label_df, fmt="d", cmap="YlGnBu")
# plt.title('标签分布热力图')
plt.xlabel('Clients')
plt.xticks(range(0, 100, 10), [str(i) for i in range(0, 100, 10)])
plt.ylabel('Labels')
plt.show()